<a href="https://colab.research.google.com/github/studam/Covid-19-fake-news/blob/main/Covid_19database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"
# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 252 kB in 2s (112 kB/s)
Reading package li

In [37]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-05-06 08:00:11--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.52MB/s    in 0.2s    

2022-05-06 08:00:11 (5.52 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [39]:
import pandas as pd

In [40]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Covid-19database").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [46]:
from pyspark import SparkFiles
url ="https://covid19needed.s3.eu-west-2.amazonaws.com/fakenNews.csv"
spark.sparkContext.addFile(url)
fake = spark.read.csv(SparkFiles.get("fakeNews.csv"), sep=",", header=True)
# Show DataFrame
fake.show()

+-----------+--------------------+--------------------+-------------+--------------+--------------------+------------------+--------------------+-----------------+-------------+------------+
|Date Posted|                Link|                Text|       Region|       Country|         Explanation|            Origin|          Origin_URL|  Fact_checked_by|Poynter_Label|Binary Label|
+-----------+--------------------+--------------------+-------------+--------------+--------------------+------------------+--------------------+-----------------+-------------+------------+
|     2/7/20|https://www.poynt...| Tencent revealed...|       Europe|        France| The screenshot i...|           Twitter|https://www.liber...|        CheckNews|   Misleading|           0|
|     2/7/20|https://www.poynt...| Taking chlorine ...|       Europe|       Germany| Chlorine dioxide...|           Website|https://correctiv...|        Correctiv|        FALSE|           0|
|     2/7/20|https://www.poynt...| This video

In [47]:
from pyspark import SparkFiles
url ="https://covid19needed.s3.eu-west-2.amazonaws.com/true.nNews.csv"
spark.sparkContext.addFile(url)
true = spark.read.csv(SparkFiles.get("trueNews.csv"), sep=",", header=True)
# Show DataFrame
true.show()

+--------------------+--------------------+--------------------+----------+----------+----------+-----+
|         Date Posted|                Link|                Text|    Region|  Username| Publisher|Label|
+--------------------+--------------------+--------------------+----------+----------+----------+-----+
|             2/11/20|https://twitter.c...|Just in: Novel co...|     India| the_hindu| The Hindu|    1|
|             2/12/20|https://twitter.c...|WHO officially na...|      null|      null|      null| null|
| https://www.ndtv...|                null|                null|      null|      null|      null| null|
|#CoronavirusOutbr...|               India|                ndtv|      NDTV|         1|      null| null|
|             2/12/20|https://twitter.c...|"The #UN #health ...|      null|      null|      null| null|
|#coronavirus http...|               India|           the_hindu| The Hindu|         1|      null| null|
|             2/14/20|https://twitter.c...|The Indian Embass...|

In [48]:
# Count rows fake
fake.count()

3796

In [49]:
#Count rows true
true.count()


6004

In [50]:
# Clean up DataFrames to match tables
#pyspark
from pyspark.sql.functions import to_date
# Review DataFrame
review_fake = fake.select(["Date Posted", "Text", "Region", "Origin", "Binary Label"])
#Change column name

review_fake.withColumnRenamed("Binary Label","Label").printSchema()

root
 |-- Date Posted: string (nullable = true)
 |-- Text: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Label: string (nullable = true)



In [51]:
review_fake.show()

+-----------+--------------------+-------------+------------------+------------+
|Date Posted|                Text|       Region|            Origin|Binary Label|
+-----------+--------------------+-------------+------------------+------------+
|     2/7/20| Tencent revealed...|       Europe|           Twitter|           0|
|     2/7/20| Taking chlorine ...|       Europe|           Website|           0|
|     2/7/20| This video shows...|        India|          Facebook|           0|
|     2/7/20| The Asterix comi...|        India|           Twitter|           0|
|     2/7/20| Chinese Presiden...|        India|          Facebook|           0|
|     2/7/20| China seeks cour...|United States|                ab|           0|
|     2/8/20| A video shows pe...|        India|           Twitter|           0|
|     2/8/20| A video says wee...|        India| Twitter, WhatsApp|           0|
|     2/8/20| Coronavirus has ...|        India|          Facebook|           0|
|     2/8/20| In a photo of 

In [52]:
# Clean up DataFrames to match tables
#pyspark
from pyspark.sql.functions import to_date
# Review DataFrame
review_true = true.select(["Date Posted", "Text", "Region", "Label"])
review_true.show()


+--------------------+--------------------+----------+-----+
|         Date Posted|                Text|    Region|Label|
+--------------------+--------------------+----------+-----+
|             2/11/20|Just in: Novel co...|     India|    1|
|             2/12/20|WHO officially na...|      null| null|
| https://www.ndtv...|                null|      null| null|
|#CoronavirusOutbr...|                ndtv|      NDTV| null|
|             2/12/20|"The #UN #health ...|      null| null|
|#coronavirus http...|           the_hindu| The Hindu| null|
|             2/14/20|The Indian Embass...|      null| null|
| https://www.indi...|          indiatoday|IndiaToday| null|
|             2/15/20|Ground Zero | How...|      null| null|
|#coronavirus #Ker...|           the_hindu| The Hindu| null|
|             2/15/20|An elderly Chines...|      null| null|
| https://trib.al/...|           the_hindu| The Hindu| null|
|             2/16/20|The total number ...|      null| null|
| https://www.indi...|  

In [53]:
review_true.printSchema()



review_true.na.drop("all").show= False


root
 |-- Date Posted: string (nullable = true)
 |-- Text: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Label: string (nullable = true)



In [54]:
review_true.show()

+--------------------+--------------------+----------+-----+
|         Date Posted|                Text|    Region|Label|
+--------------------+--------------------+----------+-----+
|             2/11/20|Just in: Novel co...|     India|    1|
|             2/12/20|WHO officially na...|      null| null|
| https://www.ndtv...|                null|      null| null|
|#CoronavirusOutbr...|                ndtv|      NDTV| null|
|             2/12/20|"The #UN #health ...|      null| null|
|#coronavirus http...|           the_hindu| The Hindu| null|
|             2/14/20|The Indian Embass...|      null| null|
| https://www.indi...|          indiatoday|IndiaToday| null|
|             2/15/20|Ground Zero | How...|      null| null|
|#coronavirus #Ker...|           the_hindu| The Hindu| null|
|             2/15/20|An elderly Chines...|      null| null|
| https://trib.al/...|           the_hindu| The Hindu| null|
|             2/16/20|The total number ...|      null| null|
| https://www.indi...|  

In [55]:
#Count rows true
true.count()


6004

In [56]:
# Products DataFrame
publisher_df = true.select(["Username", "Publisher"])
publisher_df.show()

+----------+----------+
|  Username| Publisher|
+----------+----------+
| the_hindu| The Hindu|
|      null|      null|
|      null|      null|
|         1|      null|
|      null|      null|
|         1|      null|
|      null|      null|
|         1|      null|
|      null|      null|
|         1|      null|
|      null|      null|
|         1|      null|
|      null|      null|
|         1|      null|
|indiatoday|IndiaToday|
|      null|      null|
|         1|      null|
|      null|      null|
|         1|      null|
|      null|      null|
+----------+----------+
only showing top 20 rows



In [57]:
link_df = true.select(["Link", "Text"])
link_df.show()

+--------------------+--------------------+
|                Link|                Text|
+--------------------+--------------------+
|https://twitter.c...|Just in: Novel co...|
|https://twitter.c...|WHO officially na...|
|                null|                null|
|               India|                ndtv|
|https://twitter.c...|"The #UN #health ...|
|               India|           the_hindu|
|https://twitter.c...|The Indian Embass...|
|               India|          indiatoday|
|https://twitter.c...|Ground Zero | How...|
|               India|           the_hindu|
|https://twitter.c...|An elderly Chines...|
|               India|           the_hindu|
|https://twitter.c...|The total number ...|
|               India|          indiatoday|
|https://twitter.c...|COVID-19 coronavi...|
|https://twitter.c...|Covid-19 Updates:...|
|               India|          indiatoday|
|https://twitter.c...|#Coronavirus | #L...|
|               India|           the_hindu|
|https://twitter.c...|Serum Inst

In [58]:
# Reviews DataFrame
country_df = fake.select(["Region", "Country"])
country_df.show()

+-------------+--------------+
|       Region|       Country|
+-------------+--------------+
|       Europe|        France|
|       Europe|       Germany|
|        India|         India|
|        India|         India|
|        India|         India|
|United States| United States|
|        India|         India|
|        India|         India|
|        India|         India|
|        India|         India|
|United States| United States|
|United States| United States|
|       Europe|     Lithuania|
|        India|         India|
|        India|         India|
|        India|         India|
|        India|         India|
|       Europe|         Italy|
|       Europe|         Italy|
|       Europe|         Italy|
+-------------+--------------+
only showing top 20 rows



In [59]:
# Reviews DataFrame
text_df = fake.select(["Text", "Explanation"])
text_df.show()

+--------------------+--------------------+
|                Text|         Explanation|
+--------------------+--------------------+
| Tencent revealed...| The screenshot i...|
| Taking chlorine ...| Chlorine dioxide...|
| This video shows...| A video shows ba...|
| The Asterix comi...| Coronavirus has ...|
| Chinese Presiden...| Chinese Presiden...|
| China seeks cour...| There is no evid...|
| A video shows pe...| The video shows ...|
| A video says wee...| The news clip wa...|
| Coronavirus has ...|  The claim is fake.|
| In a photo of Pa...| The image was ed...|
| Autopsy reveals ...| There is no sour...|
|" The new coronav...| There is no evid...|
| Coronavirus is c...| There is no proo...|
| Can cannabis cur...| There is no gove...|
| A woman infected...| The woman in the...|
| Garlic water can...| Garlic may have ...|
| Coronavirus is f...| There is no evid...|
| Bill Gates predi...| Bill Gates didn'...|
| Bill Gates predi...| Bill Gates did n...|
| In 2012, a Faceb...| The Faceb

In [60]:
# Push to AWS RDS instance.
mode = "append"
jdbc_url = "jdbc:postgresql://covid19.cnssxwecu01b.eu-west-2.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", "password": "Forest22", "driver":"org.postgresql.Driver"}

In [61]:
# Write publisher_df to table in RDS
publisher_df.write.jdbc(url=jdbc_url, table='publisher', mode=mode, properties=config)

In [62]:
# Write link_df to table in RDS
link_df.write.jdbc(url=jdbc_url, table='link', mode=mode, properties=config)

In [63]:
# Write country_df to table in RDS
country_df.write.jdbc(url=jdbc_url, table='country', mode=mode, properties=config)

In [64]:
# Write text_df to table in RDS
text_df.write.jdbc(url=jdbc_url, table='text', mode=mode, properties=config)